## Maaslin 2 Analysis of GO Terms for COVIRT19

 Lets install some R packages that we are gonna need to run this analysis

#if(!requireNamespace("BiocManager", quietly = TRUE))
#install.packages("BiocManager")

In [1]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#BiocManager::install(c("Maaslin2", "DESeq2",'phyloseq','microbiome','DirichletMultinomial','GenomicRanges'))

In [2]:
#install.packages("remotes")
#remotes::install_github("mikemc/speedyseq")
#install.packages(c('circlize','ggpubr','viridis','mosaic'))

Now lets load our libraries and set out environment

In [3]:
library(tidyverse)
library(phyloseq)
library(microbiome)
library(DESeq2)
library(Maaslin2)
library(parallel)
library(DirichletMultinomial)
library(pheatmap)
library(ggpubr)
library(viridis)
library(mosaic)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


microbiome R package (microbiome.github.com)
    


 Copyright (C) 2011-2020 Leo Lahti, 
    Sudarshan Shetty et al. <microbiome.github.io>



Attaching package: ‘microbiome’


The following object is masked from ‘package:ggplot2’:

    alpha


The following object is masked from ‘package:base’:

    transform


Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEv

In [4]:
setwd('/home/jovyan/work/Aagaard_Raid3/microbial/GO_term_analysis/')
getwd()

[1] "/home/jovyan/work/Aagaard_Raid3/microbial/GO_term_analysis"

In [5]:
raw<-as_tibble(read.table("Combined_BALF_GO_Terms_parent_propagated.tsv", sep = "\t", row.names = NULL, header = T, quote = "", comment.char = ""))

A tibble: 47,233 x 2,020     # good so far now
do a little regex and fix some stuff

In [6]:
colnames(raw)<-gsub("NA_tax","unclass", colnames(raw))%>%str_replace_all("NC1_SRR7796663", "NC1.SRR7796663")

Transform the raw table by type of count (euk, term, bac, arc)

In [7]:
df<-raw %>%
  select(GO_term,namespace,depth,name,ends_with("_counts"))%>%
  pivot_longer(cols = -c(GO_term,namespace,depth,name),
               names_to =  c("sample","type","abund"),#c("Total", "Archaea","Bacteria","Eukarya", "Viridae", "Unclassified"),
               names_pattern = "(.*)_(.*)_(.*)")%>%
  select(-abund)%>%
  filter(value>1)%>%
  pivot_wider(names_from = sample, values_from=value, values_fill=0)
#SIDE NOTE:There are multiple processes and values for a single sample so you cant convert the sample to columns

Make individual tibbles for biological processes and molecular fxn

In [8]:
bio<-filter(df, namespace=="biological_process")
mol<-filter(df, namespace=="molecular_function")

make individual tibbles for each type (bac, euk, term, arc, vir, etc)

In [9]:
bio_bac<-bio%>%filter(type=="bac")%>%select(-type)
bio_term<-bio%>%filter(type=="term")%>%select(-type)
mol_bac<-mol%>%filter(type=="bac")%>%select(-type)
mol_term<-mol%>%filter(type=="term")%>%select(-type)

subselect tibbles for only the counts and go terminology

In [10]:
bio_bac_counts<-bio_bac%>%select(-c(namespace,depth,name))
bio_bac_tax<-bio_bac%>%select(GO_term,namespace,depth,name)
mol_bac_counts<-mol_bac%>%select(-c(namespace,depth,name))
mol_bac_tax<-mol_bac%>%select(GO_term,namespace,depth,name)

convert them to dataframes for downstream import to phylsoeq

In [11]:
bio_bac_counts<-data.frame(bio_bac_counts, row.names=1)
bio_bac_tax<-data.frame(bio_bac_tax, row.names=1)
mol_bac_counts<-data.frame(mol_bac_counts, row.names=1)
mol_bac_tax<-data.frame(mol_bac_tax, row.names=1)

convert the dataframes into phyloseq formats

In [12]:
bio_bac_counts_phy <- otu_table(bio_bac_counts, taxa_are_rows=TRUE)
bio_bac_tax_phy <- tax_table(as.matrix(bio_bac_tax), errorIfNULL=TRUE)
mol_bac_counts_phy<-otu_table(mol_bac_counts, taxa_are_rows = T)
mol_bac_tax_phy<-tax_table(as.matrix(mol_bac_tax), errorIfNULL = T)

import your metadata

In [13]:
bio_bac_sam<-as.data.frame(read.table("Combined_BALF_GO_Terms_metadata2.txt",header = T, sep = "\t",row.names = 1))

a little regex to fix the stupid filename

In [14]:
rownames(bio_bac_sam)<-rownames(bio_bac_sam)%>%str_replace_all("NC1_SRR7796663", "NC1.SRR7796663")
bio_bac_sam$accession<-rownames(bio_bac_sam)

I want to take a moment and thank the curators for all their hard work in annotation the metadata........ =)

In [15]:
bio_bac_sam$outcome<-bio_bac_sam$outcome%>%
str_replace_all("recovered", "Recovered")%>%
str_replace_all("deceased","Deceased")%>%
str_replace_all('stabilized',"Stabilized")
###DONT FORGET TO DELETE THESE LINES LATER AFTER YOUR DONE PLAYING AROUND
#str_replace_all('Stabilized',"Survived")%>%
#str_replace_all("Recovered", "Survived")
###############################################

In [16]:
bio_bac_sam$sex<-bio_bac_sam$sex%>%
str_replace_all("M", "male")%>%
str_replace_all("F", "female")%>%
str_replace_all("na", "<NA>")
tally(~sex,bio_bac_sam)

sex
  <NA> female   male   <NA> 
    22     54     61     30 

In [17]:
tally(~case,bio_bac_sam,format = "data.frame")
tally(~sample_type,bio_bac_sam,format = "data.frame")
tally(~outcome,bio_bac_sam, format = "data.frame")

case,Freq
<fct>,<int>
Community_acquired_pneumonia,25
Control_Healthy,32
Control_Neg,5
Control_Sick,36
Control_Unknown,21
COVID19,48


sample_type,Freq
<fct>,<int>
Asthma,7
Asthma_Ex_smoker,3
Asthma_Smoker,2
Community_acquired_pneumonia,25
COVID_19,48
Healthy,32
neg_control,5
Obese,6
Obese_Asthma,9


outcome,Freq
<fct>,<int>
Deceased,20
Recovered,11
Stabilized,8
NA,128


making physeq object

In [18]:
bio_bac_pseq <- phyloseq(bio_bac_counts_phy, bio_bac_tax_phy, sample_data(bio_bac_sam))
mol_bac_pseq<-phyloseq(mol_bac_counts_phy,mol_bac_tax_phy, sample_data(bio_bac_sam))
bac_pseq<-merge_phyloseq(bio_bac_pseq,mol_bac_pseq)
bac_pseq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 13846 taxa and 167 samples ]
sample_data() Sample Data:       [ 167 samples by 71 sample variables ]
tax_table()   Taxonomy Table:    [ 13846 taxa by 3 taxonomic ranks ]

In [19]:
filtme<-c("GO:0003674")
bac_pseq <- prune_taxa(taxa=taxa_names(bac_pseq)!=filtme, bac_pseq)
filtme<-c("GO:0008150")
bac_pseq <- prune_taxa(taxa=taxa_names(bac_pseq)!=filtme, bac_pseq)
bac_pseq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 13844 taxa and 167 samples ]
sample_data() Sample Data:       [ 167 samples by 71 sample variables ]
tax_table()   Taxonomy Table:    [ 13844 taxa by 3 taxonomic ranks ]

filter out the negative control and unknown samples

In [20]:
bac_pseq_no_neg<-subset_samples(bac_pseq, sample_type!="neg_control")
bac_pseq_no_neg# [ 13846 taxa and 162 samples ]:
bac_pseq_no_neg<-subset_samples(bac_pseq_no_neg, sample_type!="Unknown")
bac_pseq_no_neg# [ 13846 taxa and 141 samples ]:

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 13844 taxa and 162 samples ]
sample_data() Sample Data:       [ 162 samples by 71 sample variables ]
tax_table()   Taxonomy Table:    [ 13844 taxa by 3 taxonomic ranks ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 13844 taxa and 141 samples ]
sample_data() Sample Data:       [ 141 samples by 71 sample variables ]
tax_table()   Taxonomy Table:    [ 13844 taxa by 3 taxonomic ranks ]

Lets change the names of the Go Terms so we can understand the description as well as the tag

This code was causing mismatches with name and GO TAG and has since been resolved 19 NOV 2020

In [21]:
#names<-paste(taxa_names(bac_pseq_no_neg),get_taxa_unique(bac_pseq_no_neg,taxonomic.rank = "name" ),sep = "-")
#taxa_names(bac_pseq_no_neg)<-names

NEW and improved code

In [22]:
tax<-data.frame(tax_table(bac_pseq_no_neg))
names<-paste(rownames(tax),tax$name,sep="-")
length(names)
taxa_names(bac_pseq_no_neg)<-names

[1] 13844

# DESeq2 VST transformation

In [23]:
sample_info_tab<-sample_data(bac_pseq_no_neg)
sample_info_tab_phy <- sample_data(sample_info_tab)
deseq_counts<-phyloseq_to_deseq2(physeq = bac_pseq_no_neg,design = ~ 1) 
deseq_counts_vst <- estimateSizeFactors(deseq_counts, type = "poscounts")
vst_trans_count_tab <- assay(deseq_counts_vst)

converting counts to integer mode



#YAAAAAAAAAAAAAAAASSSSSSSSSSSSS THANK YOU LIMMMA

Dont worry about the limma batch effect correction step, I think I found a better way by including it in the multivariate model 

In [24]:
#vst_trans_count_tab2 <- limma::removeBatchEffect(vst_trans_count_tab, sample_info_tab$publication)

IT FIXED THE BATCH EFFECT!

### convert the normalized counts to a phyloseq object and transform into relative abundances

In [25]:
vst_count_phy <- otu_table(vst_trans_count_tab, taxa_are_rows=T)
vst_tax_phy <- tax_table(bac_pseq_no_neg)
vst_physeq <- phyloseq(vst_count_phy, vst_tax_phy,sample_data(bac_pseq_no_neg))
vst_physeq_comp<-microbiome::transform(x = vst_physeq,transform = "compositional")

# MaAsLIN2

In [26]:
#dir.create("R_Maaslin2") # Create a new directory
setwd("/home/jovyan/work/Jochum_3/jupyter_lab/GO_term_analysis/R_Maaslin2/ag1") # Change the current working directory 
getwd() #check if directory has been successfully changed

ERROR: Error in setwd("/home/jovyan/work/Jochum_3/jupyter_lab/GO_term_analysis/R_Maaslin2/ag1"): cannot change working directory


In [ ]:
df_input_data2<-data.frame(t(otu_table(vst_physeq_comp)))
df_input_metadata2<-data.frame(sample_data(vst_physeq_comp))

In [ ]:
#class(df_input_metadata2$case)
#df_input_metadata2$age<-as.factor(df_input_metadata2$age)
#df_input_metadata2$temp_degC<-as.factor(df_input_metadata2$temp_degC)
#df_input_metadata2$days.after.onset<-as.factor(df_input_metadata2$days.after.onset)
#df_input_metadata2$case<-factor(x = df_input_metadata2$case, levels = c("COVID19","Community_acquired_pneumonia","Control_Sick","Control_Healthy"))
#df_input_metadata2$outcome<-factor(x = df_input_metadata2$outcome, levels = c("Deceased","Stabilized","Recovered"))
#class(df_input_metadata2$case)
#df_input_metadata2$case

## ok so here are the parameters you want to manipulate:
min abundance= the min rel abund hits (1%) #filters out XXXX GO_terms \
min prevalence = Min samples required with min abundance for a feature not to be filtered (0.1=10%=14.1000 samples) \
max_significance = the maximinum p adjusted value to be significant \

This will filter out 13770 GO TERMS \

#_normalization = CLR transformation_ \
##### CORRECTION dont normalize here, just use the VST transformed counts
correction = the mutliple test correction method to be done (BH=Benjamini-Hochberg)

In [ ]:
case<-Maaslin2(
  input_data = df_input_data2,
  input_metadata = df_input_metadata2,
  output="./case",
  min_abundance = 0.01,
  min_prevalence = 0.01,
  normalization = "NONE",
  transform = "NONE",
  analysis_method = "LM",
  max_significance = 0.25,
  #random_effects = c("sample_name","publication","collection_location","sequence_type"),
  random_effects = c("sample_name","publication"),
  fixed_effects = c("case"),
  correction="BH",
  standardize = TRUE,
  cores = 48,
  plot_heatmap = TRUE,
  plot_scatter = TRUE,
  heatmap_first_n = 100,
  reference=c("case,COVID19"))

In [ ]:
outcome<-Maaslin2(
  input_data = df_input_data2,
  input_metadata = df_input_metadata2,
  output="./outcome",
  min_abundance = 0.001,
  min_prevalence = 0.001,
  normalization = "NONE",
  transform = "NONE",
  analysis_method = "LM",
  max_significance = 0.25,
  #random_effects = c("sample_name","publication","collection_location","sequence_type"),
  random_effects = c("sample_name","publication"),
  fixed_effects = c("outcome"),
  correction="BH",
  standardize = TRUE,
  cores = 48,
  plot_heatmap = TRUE,
  plot_scatter = TRUE,
  heatmap_first_n = 100,
  reference=c("outcome,Deceased"))

In [ ]:
#I need to figure out how to change how to pivot wider the case colum for an age analysis
df_input_metadata2$age<-as.numeric(df_input_metadata2$age)

age<-Maaslin2(
  input_data = df_input_data2,
  input_metadata = df_input_metadata2,
  output="./age",
  min_abundance = 0.001,
  min_prevalence = 0.001,
  normalization = "NONE",
  transform = "NONE",
  analysis_method = "LM",
  max_significance = 0.25,
  #random_effects = c("sample_name","publication","collection_location","sequence_type"),
  random_effects = c("sample_name","publication"),
  fixed_effects = c("case","age"),
  correction="BH",
  standardize = TRUE,
  cores = 48,
  plot_heatmap = TRUE,
  plot_scatter = TRUE,
  heatmap_first_n = 100,
  reference=c("case,COVID19"))

## MaAsLin2 outcome analyisis (pruned_samples)

In [ ]:
bac_pseq_outcome<-subset_samples(physeq = bac_pseq_no_neg,outcome!="NA")
#bac_pseq_outcome<-subset_samples(physeq = bac_pseq_no_neg, case=="COVID19")
bac_pseq_outcome
sample_info_tab<-sample_data(bac_pseq_outcome)
sample_info_tab_phy <- sample_data(sample_info_tab)
deseq_counts<-phyloseq_to_deseq2(physeq = bac_pseq_outcome,design = ~ 1) 
deseq_counts_vst <- estimateSizeFactors(deseq_counts, type = "poscounts")
vst_trans_count_tab <- assay(deseq_counts_vst)
vst_count_phy <- otu_table(vst_trans_count_tab, taxa_are_rows=T)
vst_tax_phy <- tax_table(bac_pseq_no_neg)
vst_physeq <- phyloseq(vst_count_phy, vst_tax_phy,sample_data(bac_pseq_outcome))
vst_physeq_comp<-microbiome::transform(x = vst_physeq,transform = "compositional")
df_input_data2<-data.frame(t(otu_table(vst_physeq_comp)))
df_input_metadata2<-data.frame(sample_data(vst_physeq_comp))

#### ok thats interesting it looks like it just through everything else into NA regardless of case... I guess lets trying it again with only covid cases and also look at age

In [ ]:
df_input_metadata2$age<-as.numeric(df_input_metadata2$age)
subset_outcome<-Maaslin2(
  input_data = df_input_data2,
  input_metadata = df_input_metadata2,
  output="./subset_outcome",
  min_abundance = 0.001,
  min_prevalence = 0.001,
  normalization = "NONE",
  transform = "NONE",
  analysis_method = "LM",
  max_significance = 0.25,
  random_effects = c("sample_name","publication"),
  fixed_effects = c("age","outcome"),
  correction="BH",
  standardize = TRUE,
  cores = 48,
  plot_heatmap = TRUE,
  plot_scatter = TRUE,
  heatmap_first_n = 100,
  reference="outcome,Deceased")

## DMM modeling using the MaAslin2 derived terms

Ok, lets import the MaAsLin2 derived significant terms

In [ ]:
#sig<-read.table("Significant_Go_terms_for_Beth_Maaslin2.txt",header = T,sep = "\t")
#head(sig)

## DMM Preprocessing /filtering

Ok lets filter out the GO_tag mataches from our phyloseq object

In [ ]:
Terms<-sig$GO_Tag

In [ ]:
bac_pseq_no_neg<-subset_samples(bac_pseq, sample_type!="neg_control")
bac_pseq_no_neg<-subset_samples(bac_pseq_no_neg, sample_type!="Unknown")
bac_pseq_no_neg# [ 13846 taxa and 141 samples ]:

In [ ]:
bac_pseq_prune<-prune_taxa(x = bac_pseq_no_neg,taxa = Terms)
bac_pseq_prune #[ 92 taxa and 141 samples ]
#bac_pseq_prune <- prune_samples(sample_sums(bac_pseq_prune) > 1, bac_pseq_prune)
#bac_pseq_prune#[ 92 taxa and 141 samples ]
#bac_pseq_prune <- prune_taxa(taxa_sums(bac_pseq_prune) > 1, bac_pseq_prune)
#bac_pseq_prune #[ 92 taxa and 141 samples ]

ok lets rename our GO_terms again

In [ ]:
tax<-data.frame(tax_table(bac_pseq_prune))
names<-paste(rownames(tax),tax$name,sep="-")
length(names)
taxa_names(bac_pseq_prune)<-names

filter out the depth 0 mol fxn and bio proc, empty GO Terms, and empty samples

#### DMM modeling time

convert counts to a matrix

In [ ]:
dat <- abundances(bac_pseq_prune)
count <- as.matrix(t(dat))

Fit the dmm model

In [ ]:
fit <- mclapply(1:8, dmn, count = count, verbose=TRUE)

Check the model fit with different number of mixture componenets using standard information criteria

In [ ]:
lplc <- sapply(fit, laplace) # AIC / BIC / Laplace
aic  <- sapply(fit, AIC) # AIC / BIC / Laplace
bic  <- sapply(fit, BIC) # AIC / BIC / Laplace
plot(lplc, type="b", xlab="Number of Dirichlet Components", ylab="Model Fit")

In [ ]:
#identify the number of clusters that best fits the model

In [ ]:
best <- fit[[which.min(lplc)]]
best <-fit[[3]]
best

In [ ]:
#save.image(file = "go_terms_dmm.rdata")

#make a heatmap visualization of the cluster

log 2 Heatmap

In [ ]:
heatmapdmn(count, fit[[1]], best,ntaxa = 50,
           transform =log2, lblwidth = 0.2 * nrow(count))

square root version

In [ ]:
heatmapdmn(count, fit[[1]], best,ntaxa = 50,
           transform =sqrt, lblwidth = 0.2 * nrow(count))

print out the theta values

In [ ]:
mixturewt(best)

save the datasheet that show which GO terms contributed to each dmm group

In [ ]:
write.table(fitted(best),"GO_TERMS_DMM_contributions.tsv", sep="\t")

save a datasheet that identifies which sample belongs to which dmm group

In [ ]:
ass <- apply(mixture(best), 1, which.max)
write.table(ass,"GO_TERMS_DMM_groups.tsv",sep="")

In [ ]:
#add the dmm group to the metadata
sample_data(bac_pseq_prune)$dmn<-ass
bac_pseq_prune_comp<-microbiome::transform(bac_pseq_prune,"compositional")
#melt the phyloseq object into tidy form
tmp<-psmelt(bac_pseq_prune_comp)
tmp<-as_tibble(tmp)

In [ ]:
gghistogram(tmp,x = "Abundance",y = "..count..")+scale_x_log10() #move each dmm group into a colum of its own

In [ ]:
#tmp$log2Abundance<-log2(tmp$Abundance)

In [ ]:
#subset the dataset to only include the case, Go_term, count, and dmm group.
#obtain the avergage count for each Go term
#order the go terms from hight to lowest count

I added these filtering commands to pull out the counts with less than 1% relabund or greater tahn 22% (ie:molecular function)

In [ ]:
d2<-tmp %>%
  select(case,OTU,Abundance, dmn)%>%
  group_by(OTU,case, dmn) %>%
  summarise(avg = mean(Abundance)) %>%
  arrange(desc(avg))

In [ ]:
#d2$avg<-sqrt(d2$avg)

In [ ]:
d3<-tidyr::spread(d2,dmn, avg)

In [ ]:
#get the total count of the go terms and oder from greates to lowest

In [ ]:
d3<-tidyr::spread(d2,dmn,avg)
d3$tot<-rowSums(d3[3:5], na.rm = T)
d3<-d3%>%arrange(desc(tot))
d3$tot<-NULL
head(d3)

In [ ]:
d3<-d3%>%gather(data = d3,avg,3:5)
colnames(d3)<-c("name","case", "dmn","avg")

make the balloon plot

In [ ]:
d4<-d3%>%filter(avg>0.01)%>%arrange(name,case,dmn)
d4<-d4[1:108,]

In [ ]:
my_pal<-viridis(n = 256, alpha = 1, begin = 0, end = 1, direction = 1)

In [ ]:
options(repr.plot.width=14, repr.plot.height=10)
a<-ggplot(data = d4,mapping = aes(x = factor(dmn),y =reorder(name,avg),size=avg,color=avg))+
geom_point()+
#theme(text=element_text(size=20))+
scale_colour_gradientn(colours = my_pal,trans="log2")+
facet_grid(facets = ~ case)+
theme_minimal(base_size = 16)

a


In [ ]:
#    ggballoonplot(d4, y ="name",x = "dmn", size = "avg", facet.by = "case",fill = "avg",ggtheme = theme_minimal())+
#      guides(size = FALSE)+
#    font("y.text", size = 12)+scale_fill_viridis_c()

In [ ]:
#save.image("GO_TERM_Maaslin2_19_NOV_2020.rda")
#load.Rdata("GO_TERM_Maaslin2_19_NOV_2020.rda")

## DATA VISUALIZATION TIME and outcome comparison time

In [ ]:
sam<-as_tibble(sample_data(bac_pseq_no_neg))
tally(x = case~outcome,sam)
chisq.test(~outcome,sam)

ok lets do a binomial test to see if we can use this

In [ ]:
rec <-binom.test(~ outcome=="Recovered", data = sam)
dec <-binom.test(~ outcome=="Deceased", data = sam)
sta <-binom.test(~ outcome=="Stabilized", data = sam)
out_tbl<-tally(x = case~outcome,sam)
result2 <-chisq.test(table(out_tbl))
rec
dec
sta
out_tbl
result2
chisq(result2)

YAAAASSSSSS CASE ~OUTCOME Chi-squared TEST P=0.0517
LESGOOOO

That made the results even less significant for some reason!  ok lets try with 